# Feature Selection

대규모 데이터를 기반으로 분류모델을 생성하는 경우 샘플의 갯수 혹은 피처의 갯수가 방대해 이슈가 발생할 수 있다. 아래는 그 중 독립변수가 방대해서 문제가 발생하는 경우 문제 접근 방법에 대해 설명한다.

다음의 데이터는 독립변수가 47236개, 샘플사이즈가 23149개이다.

In [3]:
%%time
from sklearn.datasets import fetch_rcv1
rcv_train = fetch_rcv1(subset="train")
rcv_test = fetch_rcv1(subset="test")
X_train = rcv_train.data
y_train = rcv_train.target
X_test = rcv_test.data
y_test = rcv_test.target

# Ont-Hot-Encoding된 라벨을 정수형으로 복원
classes = np.arange(rcv_train.target.shape[1])
y_train = y_train.dot(classes)
y_test = y_test.dot(classes)

print(X_train.shape)

(23149, 47236)
Wall time: 5min 55s


# 분산에 의한 선택

데이터를 분류하기 위해 중요한 독립변수는 $y$값과의 상관관계가 큰 데이터가 퍼포먼스에 좋은 영향을 미친다고 기대할 수 있다.

그런데 데이터 자체의 분산이 작아 $y$값과 상관관계가 작게 발생하는 경우 오히려 퍼포먼스에 악영향을 미칠 수 있다.

이러한 접근법으로 분산을 이용해 Feature Selection을 진행할 수 있는데 sklearn에서 이를 위한 VarianceThreshold 메소드를 제공한다. 파라미터로는 분산의 크기를 지정해 그보다 분산이 작은 피처를 제거한다.

In [17]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(1e-5)
X_train_sel = selector.fit_transform(X_train)
X_test_sel = selector.transform(X_test)
X_train_sel.shape

(23149, 14330)

분산을 이용한 Feature Selection을 진행한 결과 원본 데이터의 약 30%에 해당하는 피처만이 선택된 것을 확인할 수 있다.

이제 베르누이 나이브베이즈 분류모델을 이용해 Train set과 Test set에 대한 퍼포먼스를 측정해보자.

In [7]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [18]:
%%time
model = BernoulliNB()
model.fit(X_train, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test))))

train accuracy:0.381
test accuracy :0.324
Wall time: 22.5 s


In [21]:
%%time
model = BernoulliNB()
model.fit(X_train_sel, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))

train accuracy:0.529
test accuracy :0.441
Wall time: 18.1 s


원본데이터와 분산선택을 통한 데이터로 두 모델을 학습시켜 성능을 측정한 결과 ACC가 약 10% 증가함을 확인할 수 있다.

이처럼 피처 선택 후 성능이 좋아지는 이유는 다음과 같다.

NB 모델의 가정은 나이브 가정으로 서로 독립적이라고 가정한다. 하지만 현실의 데이터가 그렇지 못한 경우 퍼포먼스가 떨어지게 되는데 독립변수의 갯수가 적어지면서 나이브가정이 잘 맞게 되면서 성능이 좋아지는 경향을 보인다.

# 단일 변수 선택

단일 변수 선택법은 하나 하나의 독립변수만을 이용해 예측 모형의 성능을 평가한다. 이를 통해 최종적으로 분류성능 혹은 상관관계가 가장 높은 독립변수만을 선택하는 방법이다.

- chi2: 카이제곱 검정 통계값
- f_classif : ANOVA분석 F검정 통계값
- mutual_info_classif : 상호정보량(mutual information)

다만 단일 변수 선택법은 하나 하나의 독립변수로 그리디한 선택으로 동작해 예측 모형에 대한 성능을 평가할 때 상관관계가 낮은 두 독립변수가 함께 사용되었을 때의 좋은 성능을 내는 피처는 선택하지 못한다.

SelectKBest 클래스는 성능이 좋은 변수만을 사용하는 전처리기이다.

In [23]:
from sklearn.feature_selection import chi2, SelectKBest

In [25]:
%%time

# 분산선택 시 선택된 피처의 갯수를 k로 지정
selector1 = SelectKBest(chi2, k=14330)
X_train1 = selector1.fit_transform(X_train, y_train)
X_test1 = selector1.transform(X_test)

model = BernoulliNB()
model.fit(X_train1, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train1))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test1))))

train accuracy:0.505
test accuracy :0.438
Wall time: 17 s


카이제곱 검정 통계값을 이용해 피처를 선택한 결과 원본 데이터보다 10% 정도 높은 성능을 확인할 수 있다.

# Feature Importance를 이용한 변수 선택

RandomForest와 같은 모델을 이용해 Feature Importance를 계산할 수 있는데 이 정보를 이용해 중요한 피처를 선택하고 해당 피처를 다른 모형에서 사용하는 방법이다.

In [26]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [29]:
%%time
n_sample = 10000
idx = np.random.choice(range(len(y_train)), n_sample)
model_sel = ExtraTreesClassifier(n_estimators=50).fit(X_train[idx, :], y_train[idx])
selector = SelectFromModel(model_sel, prefit=True, max_features=14330)
X_train_sel = selector.transform(X_train)
X_test_sel = selector.transform(X_test)

Wall time: 27.2 s


In [30]:
%%time
model = BernoulliNB()
model.fit(X_train_sel, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))

train accuracy:0.601
test accuracy :0.491
Wall time: 17.8 s
